# 第2章: UNIXコマンド

In [1]:
!wget https://nlp100.github.io/data/popular-names.txt

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/yukimaru/.wget-hsts'. HSTS will be disabled.
--2020-04-12 11:56:17--  https://nlp100.github.io/data/popular-names.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55026 (54K) [text/plain]
Saving to: ‘popular-names.txt’

popular-names.txt   100%[===================>]  53.74K  --.-KB/s    in 0.09s   

2020-04-12 11:56:18 (599 KB/s) - ‘popular-names.txt’ saved [55026/55026]



## 10. 行数のカウント
行数をカウントせよ．確認にはwcコマンドを用いよ．

In [2]:
with open('e10.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
for i,_ in enumerate(sys.stdin,1): pass
print(i)
""")

In [3]:
%%bash
fname="popular-names.txt"
check="cat $fname| wc -l"
eval $check
res=`diff <(./e10.py < $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo $res; fi

2780
OK


## 11. タブをスペースに置換
タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [4]:
with open('e11.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
for line in sys.stdin:
    sys.stdout.write(line.replace('\t',' '))
""")

In [5]:
%%bash
fname="popular-names.txt"
check="sed -e 's/\t/ /g' $fname"
res=`diff <(./e11.py < $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo $res; fi

OK


## 12. 1列目をcol1.txtに，2列目をcol2.txtに保存
各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [6]:
with open('popular-names.txt') as fr, open('col1.txt','w') as f1, open('col2.txt','w') as f2:
    for l in fr: 
        s = l.split('\t')
        f1.write(s[0]+"\n"); f2.write(s[1]+"\n")

In [7]:
%%bash
res=`diff col1.txt <(cut -f1 popular-names.txt)`
if [ "$res" = "" ]; then echo "OK1"; else echo $res; fi
res=`diff col2.txt <(cut -f2 popular-names.txt)`
if [ "$res" = "" ]; then echo "OK2"; else echo $res; fi

OK1
OK2


## 13. col1.txtとcol2.txtをマージ
12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [8]:
with open('col1.txt') as f1, open('col2.txt') as f2, open('marged.txt','w') as fr:
    for l1,l2 in zip(f1,f2):
        fr.write(l1.rstrip()+'\t'+l2)

In [9]:
%%bash
res=`diff marged.txt <(paste col1.txt col2.txt)`
if [ "$res" = "" ]; then echo "OK"; else echo $res; fi

OK


## 14. 先頭からN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [10]:
with open('e14.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
N=int(sys.argv[1])
for i,line in enumerate(sys.stdin,1):
    sys.stdout.write(line)
    if i>=N: break\
""")

In [11]:
%%bash
fname="popular-names.txt"
N=5
check="head -n$N $fname"
res=`diff <(./e14.py $N< $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo $res; fi

OK


## 15. 末尾のN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

In [12]:
with open('e15.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
N=int(sys.argv[1])
l=max([len(sys.stdin.readline()) for _ in range(N)])
end=sys.stdin.seek(0,2)
tmp=[]; i=0
while len(tmp)<N+1:
    sys.stdin.seek(end-l*(N+i))
    tmp = sys.stdin.readlines()
    i+=1
for l in tmp[-N:]: sys.stdout.write(l)\
""")

In [13]:
%%bash
fname="popular-names.txt"
N=5
check="tail -n$N $fname"
res=`diff <(./e15.py $N< $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo $res; fi

OK


## 16. ファイルをN分割する
自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．

In [14]:
with open('e16.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys, os
N=int(sys.argv[1])
C=int(sys.argv[2])
os.makedirs('tmp', exist_ok=True)
for f in [open("tmp/split{:02}.txt".format(i),"w") for i in range(N)]:
    for i,line in enumerate(sys.stdin,1):
        f.write(line)
        if i>=C: break\
""")

In [15]:
%%bash
fname="popular-names.txt"
N=6
l=`./e10.py < $fname`
len=$[($l/$N)+($l%$N==0?0:1)]
rm -rf tmp
./e16.py $N $len <$fname
split -l$len popular-names.txt tmp/split-c -d --additional-suffix=.txt
for i in `seq -f "%02g" 0 $[N-1]`; do
    res=`diff tmp/split$i.txt tmp/split-c$i.txt`;
    if [ "$res" = "" ]; then echo "OK$i"; else echo $res; fi
done
rm -rf tmp

OK00
OK01
OK02
OK03
OK04
OK05


## 17. １列目の文字列の異なり
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはcut, sort, uniqコマンドを用いよ．

In [16]:
with open('e17.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
names = set()
for line in sys.stdin:
    names.add(line.split("\\t")[0])
names = sorted(names)
for name in names:
    sys.stdout.write(name+'\\n')\
""")

In [17]:
%%bash
fname="popular-names.txt"
check="cut -f1 $fname | sort | uniq"
res=`diff <(./e17.py < $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo $res; fi

OK


## 18. 各行を3コラム目の数値の降順にソート
各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

In [18]:
with open('e18.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
lis = []
for line in sys.stdin:
    lis.append(line)
lis = sorted(lis, key=lambda l:int(l.split()[2]),reverse=True)
for line in lis:
    sys.stdout.write(line)
    sys.stdout.flush()\
""")

In [19]:
%%bash
fname="popular-names.txt"
check="sort -k3 -r -n -s $fname"
res=`diff <(./e18.py < $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo "$res"; fi

OK


## 19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [20]:
with open('e19.py', 'w') as f:
    f.write("""\
#!/usr/bin/env python3
import sys
dic = {}
for line in sys.stdin:
    key = line.split('\t')[0]
    dic.setdefault(key,0)
    dic[key]+=1
ldic = sorted(dic.items(), key=lambda x:x[0], reverse=True)
ldic = sorted(ldic, key=lambda x:x[1], reverse=True)
for name,count in ldic:
    print(name)\
""")

In [21]:
%%bash
fname="popular-names.txt"
check="cut -f1 $fname | sort | uniq -c | sort -r | cut -b9-"
res=`diff <(./e19.py < $fname) <(eval $check)`
if [ "$res" = "" ]; then echo "OK"; else echo "$res"; fi

OK
